# 4 Unicode文本和字节序列

主要讲Unicode字符串、二进制序列，以及两者之间转换时使用的代码。
感觉日常场景较少遇到，可以速读。

# 4.2 字符问题（可略）

一个字符串就是一个字符序列。在2021年，“字符”的最佳定义是Unicode字符。因此，从Python 3的str对象中获取的元素是Unicode字符，这个对象与Python 2的unicode对象类似。

- 字符的标识，即码点，是0~1,114,111的数字（十进制），在Unicode标准中以4~6个十六进制数字表示，而且加前缀“U+”。例如，字母A的码点是U+0041，欧元符号的码点是U+20AC，高音谱号的码点是U+1D11E。在Unicode 6.3中（这是Python 3.4使用的版本），约10%的有效码点有对应的字符。

- 字符的具体表述取决于所用的编码。编码是在码位和字节序列之间转换时使用的算法。在UTF-8编码中，A（U+0041）的码位编码成单个字节\x41，而在UTF-16LE编码中编码成两个字节\x41\x00。再举个例子，欧元符号（U+20AC）在UTF-8编码中是三个字节\xE2\x82\xAC，在UTF-16LE中编码成两个字节\xAC\x20。

In [ ]:
# 编码和解码
s = "cafe"
print(len(s))
b = s.encode('utf8')
print(b)
print(len(b))
print(b.decode('utf8'))

# 4.3 字节概要
Python内置两种基本的二进制序列类型：Python 3引入的不可变bytes类型和Python 2.6添加的可变bytearray类型。各个字节的值介于0~255（含）之间。bytes或bytearray对象的各个元素是介于0~255之间的整数，而不像Python 2的str对象那样是单个的字符。

In [ ]:
cafe = bytes('cafe', encoding='utf_8')
print(cafe)
print(cafe[0])
print(cafe[:1])
cafe_arr = bytearray(cafe)
print(cafe_arr)
print(cafe_arr[-1:])

二进制序列的字面量标识法表明其中含有ASCII字符

- 十进制代码在32-126范围内的字节（从空格到波浪号~），使用ASCII字符本身。
- 制表符、换行符、回车符和\对应的字节，使用转义序列\t、\n、\r和\\。
- 如果字节序列同时包含两种字符串分隔符'和",整个序列使用'间隔，序列内的'转义为\'。
- 其他字节的值，使用十六进制转义序列（例如，\x00是空字节）。

构建bytes或bytearray对象还可以调用各自的构造方法，传入下述参数。
- 一个str对象和一个encoding关键字参数。
- 一个可迭代对象，提供0~255之间的数值。
- 一个实现了缓冲协议的对象（如bytes、bytearray、memoryview、array.array），此时把源对象中的字节序列复制到新建的二进制序列中。

# 4.4 基本的编码加码器
Python自带了超过100种编解码器（codec，encoder/decoder，编码器/解码器），用于在文本和字节之间相互转换。每个编解码器都有一个名称，如'utf_8'，并且经常有几个别名，如'utf8'、'utf-8'和'U8'。这些名称可以传给open()、str.encode()、bytes.decode()等函数的encoding参数。

In [ ]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

一些典型编码（简单了解即可）
- latin1（又名ISO8859-1） 重要，其他编码的基础
- cp1252（微软在Windows中的latin1超集）
- cp437（早期IBM PC的字符集）
- gb2312（简体中文，编码中文字符）
- utf-8 Web最常用的8位编码（重要了解）
- utf-16le UTF16位编码方案的一种形式

# 4.5 处理编码和解码问题

UnicodeEncodeError：把str转换成二进制序列时报错，目标编码没有定义某个字符。
UnicodeDecodeError：把二进制序列转换成str时报错。
如果源码的编码与预期不符，加载Python模块时还可能抛出SyntaxError。

## 4.5.1 处理UnicodeEncodeError

In [ ]:
city = 'São Paulo'
print(city.encode('utf_8'))
print(city.encode('utf_16'))

print(city.encode('iso8859_1'))

try:
    city.encode('cp437')
except UnicodeEncodeError as e:
    print(e)
# 跳过无法编码的字符，不好
print(city.encode('cp437', errors='ignore'))
# 用？替换无法编码的字符，也不太好
print(city.encode('cp437', errors='replace'))
# 把无法编码的字符替换成xml实体
print(city.encode('cp437', errors='xmlcharrefreplace'))

# 可以用str.isascii()方法检查是否有非ASCII字符，只要都是ASCII就一定可以编码成功
print(city.isascii())

## 4.5.2 处理UnicodeDecodeError

并非所有的字节序列都是有效的UTF-8或者UTF-16码点。因此如果遇到无法转换的字节序列时将抛出UnicodeDecodeError。
老的8位编码能解码任何字节序列流但是有可能生成乱码。


In [ ]:
# latin1编码单词 Montréal
octets = b'Montr\xe9al'
# cp1252是latin1的超集所以可以解码成功
print(octets.decode('cp1252'))
# iso8859_7是希腊文字符集，所以解码失败
print(octets.decode('iso8859_7'))
# koi8_r是俄文字符集，\xe9表示西里尔字母 “N 打不出来像这个大N"
print(octets.decode('koi8_r'))
try:
    # utf_8解码失败，因为\xe9不是有效的utf_8编码
    octets.decode('utf_8')
except UnicodeDecodeError as e:
    print(e)
# 用replace替换无效字符
octets.decode('utf_8', errors='replace')

## 4.5.3 加载模块时编码不符合预期抛出的SyntaxError

Python3默认使用UTF-8编码源码，Python2则默认使用ASCII。如果加载的.py模块中包含UTF-8之外的数据，而且没有声明编码，Python3会报SyntaxError错误。
可以在文件顶部加上如下注释，声明源码的编码。
```python
# coding: cp1252
```

## 4.5.4 如何找出字节序列的编码

如何找出字节序列的编码？不能，只能别人告知。
HTTP和XML等协议通常会在首部明确指明内容编码。如果字节流中有大于127的字节值，肯定不是ASCII编码。
如果b'\x00'字节经常出现，可能是16位或者32位编码。
如果b'\x20\x00'字节经常出现，可能是UTF-16LE编码。

python包Chardet可以推测字节序列的编码，支持30种编码。

## 4.5.5 BOM：有用的鬼符

BOM（Byte Order Mark，字节序标记）是位于Unicode文本开头的特殊编码。BOM的作用是声明Unicode文本的编码。

# 4.6 处理文本文件
Unicode三明治：尽早把输入（例如读取文件时）的字节序列解码成字符串，尽量晚地把字符串编码成字节序列（例如写入文件时）。
需要在多台设备或者多种场合下运行代码，打开文件时要始终明确传入encoding=参数。


In [ ]:
open('cafe.txt', 'w', encoding='utf_8').write('café')

# 4.7 为了正确比较而规范化Unicode字符串
因为Unicode有组合字符，变音符和附加到前一个字符上的其他记号，打印时作为一个整体，所以字符串比较起来很复杂。
使用unicodedata.normalize()函数，该函数的第一个参数是这4个字符串中的一个：'NFC'、'NFD'、'NFKC'和'NFKD'。第二个参数是要处理的文本。
NFC(Normalization Form C)使用最少的码点构成等价的字符串，而NFD把合成字符分解成基字符和单独的组合字符。

In [ ]:
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\N{COMBINING ACUTE ACCENT}'
print(len(s1))
print(len(s2))
print(len(normalize('NFC', s1)))
print(len(normalize('NFC', s2)))
print(len(normalize('NFD', s1)))
print(len(normalize('NFD', s2)))
print(normalize('NFC', s1) == normalize('NFC', s2))
print(normalize('NFD', s1) == normalize('NFD', s2))

用户输入默认是NFC形式。保存文本时最好使用normaize('NFC', user_text)。
另外两种感觉用处不大，这里不做记录。

## 4.7.1 大小写同一化
大小写同一化是把所有文本变成小写，再做其他转换。由str.casefold()方法实现，它与lower()方法效果类似，但是对于某些地区的特殊字符，casefold()处理得更彻底。

In [ ]:
from unicodedata import name
micro = 'μ'
print(name(micro))
micro_cf = micro.casefold()
print(name(micro_cf))

## 4.7.2 规范化文本匹配实用函数
如果需要处理多语言文本，下列函数是有用的。

In [ ]:
from unicodedata import normalize
def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() ==
            normalize('NFC', str2).casefold())

## 4.7.3 极端“规范化”：去掉变音符号

In [ ]:
import unicodedata
import string

def shave_marks(txt):
    """把所有的组合字符删除"""
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)

def shave_marks_latin(txt):
    """把拉丁基字符中所有的变音符号删除"""
    norm_txt = unicodedata.normalize('NFD', txt)
    latin_base = False
    keepers = []
    for c in norm_txt:
        # ord(c)返回一个字符的Unicode码位
        if unicodedata.combining(c) and latin_base:
            continue
        keepers.append(c)
        # 如果不是组合字符，那就是新的基字符
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)

# 4.8 Unicode文本排序
对字符串来说，排序时比较的是码点。但是一旦遇到非ASCII字符，就会出现问题。
在python中，非ASCII文本的标准排序方式是使用locale.strxfrm函数。


In [ ]:
import locale
my_locale = locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')
print(my_locale)
# 本地化比较
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
print(sorted(fruits, key=locale.strxfrm))

可以使用pyuca库，它是Unicode排序算法的纯Python实现，支持多种语言。

In [ ]:
# import pyuca
# coll = pyuca.Collator()
# fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
# print(sorted(fruits, key=coll.sort_key))

# 4.9 Unicode数据库
Unicode标准提供了一个完整的数据库，不仅包括码点与字符名称的映射，还包括各个字符的元数据以及字符之间的关系。
## 4.9.1 按名称查找字符

In [17]:
from unicodedata import name
print(name('A'))

LATIN CAPITAL LETTER A


## 4.9.2 字符的数值意义
unicodedata中有几个函数可以检查Unicode字符是不是表示数值，如果是的话还可以返回对应的数值。

In [18]:
import unicodedata
import re

re_digit = re.compile(r'\d')
sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'
for char in sample:
    print('U+%04x' % ord(char),
          char.center(6),
          're_dig' if re_digit.match(char) else '-',
          'isdig' if char.isdigit() else '-',
          'isnum' if char.isnumeric() else '-',
          format(unicodedata.numeric(char), '5.2f'),
          unicodedata.name(char),
          sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


# 4.10 支持str和bytes的双模式API
## 4.10.1 正则表达式中的str和bytes
如果使用bytes构建正则表达式，则\d和\w等模式只能匹配ASCII字符。如果使用str构建正则表达式，则这些模式可以匹配ASCII之外的Unicode字符。

In [19]:
import re
re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
            " as 1729 = 1³ + 12³ = 9³ + 10³.")
text_bytes = text_str.encode('utf_8')
print('Text', repr(text_str), sep='\n  ')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))
print('  bytes:', re_numbers_bytes.findall(text_bytes))
print('Words')
print('  str  :', re_words_str.findall(text_str))
print('  bytes:', re_words_bytes.findall(text_bytes))

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


## 4.10.2 os函数中的str和bytes
为了规避文件名不能编码的问题，os模块中所有接受文件名或者路径的函数，都支持str和bytes两种类型的参数。如果参数是str，那么系统调用会使用sys.getfilesystemencoding()返回的编码解码器自动转换参数，操作系统回显时也使用该编码解码器解码。
如果自动处理不了，那么可以传bytes参数。

In [ ]:
import os
os.listdir('.')
os.listdir(b'.')